In [3]:
import os
import random
import pandas as pd
import numpy as np
from datetime import timedelta
import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as plot
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
matplotlib.rcParams["timezone"] = "Europe/Berlin"

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning) # pd.concat() sucks

import data_process
import importlib
importlib.reload(data_process)

try:
    import seaborn
    colors = seaborn.color_palette()
except:
    colors = ["b", "g", "r", "c", "y"]

In [4]:
# create random time series from import data
da = data_process.DataAction()
nc = data_process.net_calc()

da.folder_path = r"G:\\My Drive\\docs\\Education\\University\\ALUF\\SSE\\6 sem\\Bachelor Thesis\\data\\"
keep_cols = ['DE_KN_residential1_grid_import',
             'DE_KN_residential2_grid_import',
             'utc_timestamp']
da.imp_procc("house_data.csv", keep_cols)

Number of data frame segments =  90
dfList created successfully.


In [5]:
def night_rand():
    """Create time-limited randomly sampled series"""

    # choose random list number for df
    df_rand = np.random.choice(len(da.dfList[:-1]))  # last list dropped, likely not full

    # parse random data and convert to time index, limit to single series
    rand_col = np.random.randint(0,2)  # rand pick between two load profiles (in imported data)
    ts1 = da.parse_procc(da.dfList[df_rand].iloc[:,rand_col]) 

    # limit to night time, random date
    date1 = np.random.choice(da.unique_date(ts1)[1:-2])
    night1 = da.get_night(ts1, date1).copy()

    return night1


In [6]:
def load_sgen_make(load_number=5):
    """Create time series test dataframe for any number of loads.
    load_number goes from 1 to n, number designation of load in network"""

    # targets
    night_loads = pd.DataFrame()
    list_loads = []
    night_sgens = pd.DataFrame()
    list_sgens = []

    # generate list of Series for concatenation
    for i in range(1, load_number+1):
        night1 = night_rand()
        list_loads.append(pd.Series(night1.values, name='loadh_' + str(i)))
        list_sgens.append(pd.Series([0]*night1.shape[0], name='sgen_' + str(i)))

    # merge series into data frames
    night_loads = pd.concat(list_loads, axis=1) * da.conv_fac / 1000000 # to MW
    night_loads.index = night1.index
    night_sgens = pd.concat(list_sgens, axis=1)
    night_sgens.index = night1.index

    return night_loads, night_sgens


night_loads, night_sgens=load_sgen_make(12)
night_sgens

,sgen_1,sgen_2,sgen_3,sgen_4,sgen_5,sgen_6,sgen_7,sgen_8,sgen_9,sgen_10,sgen_11,sgen_12
date_time,,,,,,,,,,,,
2016-12-26 20:00:00+01:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-12-26 20:01:00+01:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-12-26 20:02:00+01:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-12-26 20:03:00+01:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-12-26 20:04:00+01:00,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27 05:56:00+01:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-12-27 05:57:00+01:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-12-27 05:58:00+01:00,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
def sgen_rand(night_sgens, sgen_val):
    """Write sgen vals over random time window for cols"""
    sgens = night_sgens.columns

    for name in sgens:
        # writes directly to night_mw
        start, end = da.time_wind(night_sgens, 60)
        da.sgen_write(night_sgens, start, end, name, sgen_val)

    return night_sgens


night_mw = sgen_rand(night_sgens, 0.010)
night_mw

,sgen_1,sgen_2,sgen_3,sgen_4,sgen_5,sgen_6,sgen_7,sgen_8,sgen_9,sgen_10,sgen_11,sgen_12
date_time,,,,,,,,,,,,
2016-12-26 20:00:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2016-12-26 20:01:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2016-12-26 20:02:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2016-12-26 20:03:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2016-12-26 20:04:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27 05:56:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0
2016-12-27 05:57:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0
2016-12-27 05:58:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [35]:
# create net and assign load names
net = pn.ieee_european_lv_asymmetric("off_peak_1440")

for i in range(0, len(net.asymmetric_load)):
    bus_nmbr = net.asymmetric_load.bus.at[i]
    load_name = "loadh_" + str(i)
    sgen_name = "sgen_" + str(i)
    pp.create_load(net, bus_nmbr, 0, name=load_name)
    pp.create_sgen(net, bus_nmbr, 0, name=sgen_name)

# create dataset copy w/ index for timeseries
night_loads_ts = night_loads.copy()
night_loads_ts.index = range(0, night_loads.shape[0])
night_sgens_ts = night_sgens.copy()
night_sgens_ts.index = range(0, night_sgens.shape[0])

# create controllers
ds_loads = DFData(night_loads_ts)
ds_sgens = DFData(night_sgens_ts)
# con1 = ConstControl(
#     net,
#     element="sgen",
#     variable="p_mw",
#     element_index=net.sgen.index,
#     profile_name=net.sgen.name.tolist(),
#     data_source=ds_sgens
# )
con2 = ConstControl(
    net,
    element="load",
    variable="p_mw",
    element_index=net.load.index,
    profile_name=net.load.name.tolist(),
    data_source=ds_loads
)

nc.n_timesteps = night_loads.shape[0]
nc.time_steps = range(0, nc.n_timesteps)


In [36]:
net.load.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54],
           dtype='int64')

In [37]:
night_loads_ts

,loadh_1,loadh_2,loadh_3,loadh_4,loadh_5,loadh_6,loadh_7,loadh_8,loadh_9,loadh_10,loadh_11,loadh_12
0,0.000293,0.000450,0.000308,0.002549,0.000586,0.000879,0.000937,0.000293,0.000439,0.000293,0.000601,0.000293
1,0.000293,0.000300,0.000308,0.001934,0.000293,0.000439,0.000937,0.000601,0.000146,0.000293,0.000300,0.000308
2,0.000615,0.000300,0.000454,0.001934,0.000293,0.000439,0.000293,0.000308,0.000146,0.000293,0.000300,0.000308
3,0.000322,0.000601,0.000146,0.002988,0.000439,0.000762,0.000293,0.000293,0.000469,0.000293,0.000601,0.000000
4,0.000322,0.000300,0.000146,0.001055,0.000146,0.000322,0.000586,0.000601,0.000322,0.000293,0.000300,0.000293
...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.000586,0.000000,0.001348,0.000293,0.000293,0.000439,0.000586,0.000308,0.000439,0.000352,0.000150,0.000000
597,0.000293,0.000150,0.000601,0.000732,0.000586,0.000879,0.000586,0.000000,0.000732,0.000645,0.000300,0.000000
598,0.000293,0.000150,0.000601,0.000439,0.000293,0.000439,0.000586,0.000000,0.000293,0.000293,0.000150,0.000000
599,0.000586,0.000150,0.001348,0.000439,0.000293,0.000439,0.000645,0.000293,0.000293,0.000293,0.000150,0.000293


In [38]:
# create output writer to store results
path = r"C:\Users\panto\Documents\git\personal\EV_energy_comm\results"
ow = OutputWriter(net,
    time_steps=10,
    output_path=path,
    output_file_type=".xlsx")
ow.log_variable("res_line", "loading_percent")

In [39]:
run_timeseries(net, time_steps=nc.time_steps)

KeyError: "['loadh_0', 'loadh_13', 'loadh_14', 'loadh_15', 'loadh_16', 'loadh_17', 'loadh_18', 'loadh_19', 'loadh_20', 'loadh_21', 'loadh_22', 'loadh_23', 'loadh_24', 'loadh_25', 'loadh_26', 'loadh_27', 'loadh_28', 'loadh_29', 'loadh_30', 'loadh_31', 'loadh_32', 'loadh_33', 'loadh_34', 'loadh_35', 'loadh_36', 'loadh_37', 'loadh_38', 'loadh_39', 'loadh_40', 'loadh_41', 'loadh_42', 'loadh_43', 'loadh_44', 'loadh_45', 'loadh_46', 'loadh_47', 'loadh_48', 'loadh_49', 'loadh_50', 'loadh_51', 'loadh_52', 'loadh_53', 'loadh_54'] not in index"

In [ ]:
def end_results(sgen_val, iter, line_outputs, sgen_names):
    """Final boss"""
    
    # create df to store output
    col_names = line_outputs
    end_vals = pd.DataFrame(columns=col_names)
    end_times = pd.DataFrame(dtype='str', columns=col_names)

    for i in range(0, iter):
        # iterative scramble
        night_merge, load_names, sgen_names = auto_merge(iter)

        # assign generator value at random times
        night_mw = sgen_rand_2(ts=night_merge, sgen_names=sgen_names, sgen_val=0.01)
        da.night_mw = night_mw

        nc.euro_lv(night_mw)
        nc.four_loads_branched_out("res_line", "loading_percent")
        nc.four_loads_branched_run()
        nc.four_loads_branched_read_loadpct()

        nc.end_vals_step(nc.ll, end_vals)
        nc.end_times_step(nc.ll, end_times)

    return end_vals, end_times

night_merge, load_names, sgen_names = auto_merge()

# iter = 1
# sgen_val = 0.01 # typical val: 0.010 MW = 10kW
# end_vals, end_times = end_results(sgen_val, iter, load_names, sgen_names)

In [ ]:
night_merge